In [2]:
from pint import UnitRegistry
import pandas as pd


# helpef functions, constants, etc 

In [17]:
class ComponentReportClass:
    def __init__(self, depth, length, width):
        self.depth = depth
        self.length = length
        self.width = width 
        self.rebar = None
        self.rebar_temp_length = None
    # dimensional calcs   
    def calc_volume(self):
        return (self.depth).to("yards") * (self.width).to("yards") * (self.length).to("yards")
    def calc_top_face_area(self):
        return self.length * self.width 
    def calc_sides_face_area(self):
        return 2 * (self.depth,self.width ) + 2, (self.depth * self.length )
    # conversion calcs
    def to_cubic_yards(a):
        return a/27

    # basic work calcs
    def calc_subgrade_prep(self):
        bottom = self.calc_top_face_area()
        sides = self.calc_sides_face_area()
        return bottom + sides
    def calc_formwork(self):
        return self.calc_sides_face_area()
    def calc_excavation(self):
        # TODO add benching...
        return self.calc_volume()
    def calc_concrete(self):
        return self.calc_volume()* 1.08


    # rebar calcs
    def calc_rebar(self, quant, length):
        """quant and length need to be arrays """
        quant = to_list(quant)
        length = to_list(length)

        h = [a*b for a,b in zip(quant, length)]
        self.rebar = sum(h)
        return self.rebar

    def calc_rebar_template(self, pilecap=False):
        """quant and length need to be arrays """
        rebar_temp_length = 0
        if pilecap==True:
            rebar_temp_length = self.length * 2 + self.width * 2
        else:
            rebar_temp_length = self.length

        self.rebar_temp_length = rebar_temp_length
        return self.rebar_temp_length

    # reports
    def report_basic(self):
        report = {
            "subgrade": self.calc_subgrade_prep(),
            "excavation": self.calc_excavation(),
            "formwork": self.calc_formwork(),
            "rebar_template": self.rebar_temp_length if self.rebar_temp_length else 0*ft,
            "concrete": self.calc_concrete(),
            "rebar": self.rebar if self.rebar else 0*ft
        }
        return report 


    

In [4]:
u = UnitRegistry()
ft = u.feet
inc = u.inches

def to_list(a):
    a = [a] if type(a) != list else a
    return a

def calc_component_totals(components, fx):
    """components are defined components
    fx is a function that takes in a list of components """
    c_reports = [c.report_basic() for c in components]

    sum_components = {}
    for k in c_reports[0].keys():
        sum_components[k] = fx(c_reports, k)

    return sum_components

def to_df(a_dict):
    t1 = { f"{k} [{str(v.units)}]": round(v.magnitude,2) for k,v in a_dict.items()}
    return pd.DataFrame(t1, index=[0])

rebar types

In [5]:
rebar_dict = {
    "num_5": 4*ft + 9*inc
}

# Start Calcs

In [19]:
# pile caps 
pc1 = ComponentReportClass(2*ft, 4*ft, 4*ft) # reported in ft!

pc2 = ComponentReportClass(2*ft + 9*inc, 
                            17*ft + 2*inc, 
                            10*ft + 6*inc) 
pc1.calc_rebar(14, rebar_dict["num_5"]) 
pc2.calc_rebar(14+16, rebar_dict["num_5"])
pc1.calc_rebar_template(pilecap=True)
pc2.calc_rebar_template(pilecap=True)

pc = calc_component_totals([pc1, pc2], lambda list, key: 2*list[0][key] + list[1][key] )

to_df(pc)

,subgrade [foot ** 2],excavation [yard ** 3],formwork [foot ** 2],rebar_template [foot],concrete [yard ** 3],rebar [foot]
0,428.42,20.73,216.17,87.33,22.39,275.5


In [20]:
# gradebeams
gb_length = 6 * (2*ft + (6*inc + (1/8)*inc))
gb_width = gb_height = (12*inc).to("feet")
gb1 = ComponentReportClass(gb_width, gb_length,  gb_height) 
gb1.calc_rebar([6,80], [rebar_dict["num_5"], 1*ft])
gb1.calc_rebar_template(pilecap=False)
gb = calc_component_totals([gb1], lambda list, key: 2*list[0][key])
to_df(gb)

,subgrade [foot ** 2],excavation [yard ** 3],formwork [foot ** 2],rebar_template [foot],concrete [yard ** 3],rebar [foot]
0,94.38,1.12,64.25,30.12,1.21,217.0


# calc all component totals

In [12]:
# all 
def fx(list, key):
    # 2* pc1 + 1*pc2 * 2*gb1
   return 2*list[0][key] + list[1][key] + 2*list[2][key]

t = calc_component_totals([pc1, pc2, gb1], fx) # TODO update excel!!!
to_df(t)

,subgrade [foot ** 2],excavation [yard ** 3],formwork [foot ** 2],rebar_template [foot],concrete [yard ** 3],rebar [foot]
0,522.79,21.84,280.42,0,23.59,492.5


# Elevator pit 

- subgrade prep => outer surfaces of walls 
- excavation => enclosed volume + steping, (18" minimum depth, 1:2 ratio)
- formwork =>  outer + inner surfaces of walls 
- rebar template => ???
- concrete => volume of walls, minus overlap
- rebar => TODO 


In [42]:
elev_walls = {
    "depth" : 5*ft,
    "length": 9*ft + 10*inc + (1/4)*inc,
    "width": 7*ft + 2*inc + (7/8)*inc,
    "side_thickness": 1*ft,
    "floor_thickness": (10*inc).to("feet"),
}

sump_walls = {
    "depth" : 2*ft,
    "length": 4*ft,
    "width": 4*ft,
    "side_thickness": (8*inc).to("feet"),
    "floor_thickness": 3*ft,
}

# component dictionary to fill 
ep = {
    "subgrade": 0,
    "excavation": 0,
    "formwork": 0,
    "rebar_template": 0,
    "concrete": 0,
    "rebar": 0
}

In [22]:
def calc_wall_areas(d):
    wall_area = {
    "wide_sides": d["length"] * d["depth"],
    "short_sides": d["width"] * d["depth"],
    "floor": d["length"] * d["width"]
    }
    return wall_area

In [26]:
elev_wall_area = calc_wall_areas(elev_walls)
sump_wall_area = calc_wall_areas(sump_walls)

print(elev_wall_area, "\n", sump_wall_area)

{'wide_sides': <Quantity(49.2708333, 'foot ** 2')>, 'short_sides': <Quantity(36.1979167, 'foot ** 2')>, 'floor': <Quantity(71.3400608, 'foot ** 2')>} 
 {'wide_sides': <Quantity(8, 'foot ** 2')>, 'short_sides': <Quantity(8, 'foot ** 2')>, 'floor': <Quantity(16, 'foot ** 2')>}


subgrade and formwork 

In [29]:
side_walls_area = 2*elev_wall_area["wide_sides"] + 2*elev_wall_area["short_sides"] + 4*sump_wall_area["wide_sides"] 

In [49]:
# subgrade prep 
# outer wall area = side wall areas for elev and sump + sump floor area + (elev floor - sump floor) area 

subgrade_prep_area = side_walls_area + sump_wall_area["floor"] + ( elev_wall_area["floor"] - sump_wall_area["floor"])
ep["subgrade"] =  subgrade_prep_area

In [50]:
# formwork 
# formwork area = 2* side wall values... and no bottom values, technically inner formwork is smaller than outer 
formwork_area = 2*side_walls_area
ep["formwork"] = formwork_area

concrete 

In [56]:
# concrete -> calc wall volume and sum, then subtract volume of sump pit floor because it is taken out of elevator pit floor 

def calc_wall_volumes(d):
    return {
    "wide_sides": (d["length"] * d["depth"] * d["side_thickness"]).to("yard ** 3") ,
    "short_sides": (d["width"] * d["depth"] * d["side_thickness"]).to("yard ** 3"),
    "floor": (d["length"] * d["width"] * d["floor_thickness"]).to("yard ** 3")
    }

In [58]:
elev_volumes  = calc_wall_volumes(elev_walls)
sump_volumes = calc_wall_volumes(sump_walls)

print(elev_volumes, "\n", sump_volumes)

{'wide_sides': <Quantity(1.82484568, 'yard ** 3')>, 'short_sides': <Quantity(1.34066358, 'yard ** 3')>, 'floor': <Quantity(2.20185373, 'yard ** 3')>} 
 {'wide_sides': <Quantity(0.197530864, 'yard ** 3')>, 'short_sides': <Quantity(0.197530864, 'yard ** 3')>, 'floor': <Quantity(1.77777778, 'yard ** 3')>}


In [60]:
side_volumes = 2*elev_volumes["wide_sides"] + 2*elev_volumes["short_sides"] + 4* sump_volumes["wide_sides"]

floor_volumes = (elev_volumes["floor"] - sump_volumes["floor"]) + sump_volumes["floor"]

concrete_volume =  side_volumes + floor_volumes

ep["concrete"] = concrete_volume*1.08 # for errors 

excavation 

In [61]:
ep

{'subgrade': 274.2775607638889 <Unit('foot ** 2')>,
 'excavation': 0,
 'formwork': 405.87500000000006 <Unit('foot ** 2')>,
 'rebar_template': 0,
 'concrete': 9.322995702589164 <Unit('yard ** 3')>,
 'rebar': 0}